> From the PO.DAAC Cookbook, to access the GitHub version of the notebook, follow [this link](https://github.com/podaac/tutorials/blob/master/notebooks/sentinel-6/Access_Sentinel6_NRT.ipynb).

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Before-you-start" data-toc-modified-id="Before-you-start-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Before you start</a></span></li><li><span><a href="#Authentication-setup" data-toc-modified-id="Authentication-setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Authentication setup</a></span></li><li><span><a href="#Hands-off-workflow" data-toc-modified-id="Hands-off-workflow-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Hands-off workflow</a></span></li></ul></div>

# Access Sentinel-6 NRT Data

This notebook shows a simple way to maintain a local time series of Sentinel-6 NRT data using the [CMR Search API](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html). It downloads granules the ingested since the previous run to a designated data folder and overwrites a hidden file inside with the timestamp of the CMR Search request on success.

## Before you start

Before you beginning this tutorial, make sure you have an Earthdata account: [https://urs.earthdata.nasa.gov](https://urs.earthdata.nasa.gov) for the operations envionrment (most common) or [https://uat.urs.earthdata.nasa.gov](https://uat.urs.earthdata.nasa.gov) for the UAT environment.

Accounts are free to create and take just a moment to set up.

## Authentication setup

We need some boilerplate up front to log in to Earthdata Login.  The function below will allow Python
scripts to log into any Earthdata Login application programmatically.  To avoid being prompted for
credentials every time you run and also allow clients such as curl to log in, you can add the following
to a `.netrc` (`_netrc` on Windows) file in your home directory:

```
machine urs.earthdata.nasa.gov
    login <your username>
    password <your password>
```

Make sure that this file is only readable by the current user or you will receive an error stating
"netrc access too permissive."

`$ chmod 0600 ~/.netrc` 

*You'll need to authenticate using the netrc method when running from command line with [`papermill`](https://papermill.readthedocs.io/en/latest/). You can log in manually by executing the cell below when running in the notebook client in your browser.*

In [1]:
from urllib import request
from http.cookiejar import CookieJar
import getpass
import netrc


def setup_earthdata_login_auth(endpoint):
    """
    Set up the request library so that it authenticates against the given Earthdata Login
    endpoint and is able to track cookies between requests.  This looks in the .netrc file 
    first and if no credentials are found, it prompts for them.

    Valid endpoints include:
        urs.earthdata.nasa.gov - Earthdata Login production
    """
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        print('Please provide your Earthdata Login credentials to allow data access')
        print('Your credentials will only be passed to %s and will not be exposed in Jupyter' % (endpoint))
        username = input('Username:')
        password = getpass.getpass()

    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)

    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)

In [2]:
setup_earthdata_login_auth('urs.earthdata.nasa.gov')

Please provide your Earthdata Login credentials to allow data access
Your credentials will only be passed to urs.earthdata.nasa.gov and will not be exposed in Jupyter


Username: nickles
 ···········


In [3]:
import requests
from os import makedirs
from os.path import isdir, basename
from urllib.parse import urlencode
from urllib.request import urlopen, urlretrieve
from datetime import datetime, timedelta
from json import dumps, loads

## Hands-off workflow

This workflow/notebook can be run routinely to maintain a time series of NRT data, downloading new granules as they become available in CMR. 

The notebook writes/overwrites a file `.update` to the target data directory with each successful run. The file tracks to date and time of the most recent update to the time series of NRT granules using a timestamp in the format `yyyy-mm-ddThh:mm:ssZ`. 

The timestamp matches the value used for the [`created_at`](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#g-created-at) parameter in the last successful run. This parameter finds the granules created within a range of datetimes. This workflow leverages the `created_at` parameter to search backwards in time for new granules ingested between the time of our timestamp and now.

The variables in the cell below determine the workflow behavior on its initial run:

* `mins`: Initialize a new local time series by starting with the granules ingested since ___ minutes ago. 
* `cmr`: The domain of the target CMR instance, either `cmr.earthdata.nasa.gov`.
* `ccid`: The unique CMR `concept-id` of the desired collection.
* `data`: The path to a local directory in which to download/maintain a copy of the NRT granule time series.

In [4]:
cmr = "cmr.earthdata.nasa.gov"

# this function returns a concept id for a particular dataset
def get_collection(url: str=f"https://{cmr}/search/collections.umm_json", **params):
    return requests.get(url, params).json().get("items")[0]

#
# This cell accepts parameters from command line with papermill: 
#  https://papermill.readthedocs.io
#
# These variables should be set before the first run, then they 
#  should be left alone. All subsequent runs expect the values 
#  for cmr, ccid, data to be unchanged. The mins value has no 
#  impact on subsequent runs.
#

mins = 20

name = "JASON_CS_S6A_L2_ALT_LR_RED_OST_NRT_F"

ccid = get_collection(ShortName=name).get("meta").get("concept-id")

data = "resources/nrt"

The variable `data` is pointed at a nearby folder [`resources/nrt`](resources/nrt/) by default. **You should change `data` to a suitable download path on your file system.** An unlucky sequence of git commands could disappear that folder and its downloads if your not careful. Just change it.

The Python imports relevant to the workflow

**The search retrieves granules ingested during the last `n` minutes.** A file in your local data dir  file that tracks updates to your data directory, if one file exists. The CMR Search falls back on the ten minute window if not.

In [5]:
timestamp = (datetime.utcnow()-timedelta(minutes=mins)).strftime("%Y-%m-%dT%H:%M:%SZ")
timestamp

'2022-11-08T00:15:46Z'

This cell will replace the timestamp above with the one read from the `.update` file in the data directory, if it exists.

In [6]:
if not isdir(data):
    print(f"NOTE: Making new data directory at '{data}'. (This is the first run.)")
    makedirs(data)
else:
    try:
        with open(f"{data}/.update", "r") as f:
            timestamp = f.read()
    except FileNotFoundError:
        print("WARN: No .update in the data directory. (Is this the first run?)")
    else:
        print(f"NOTE: .update found in the data directory. (The last run was at {timestamp}.)")

NOTE: Making new data directory at 'resources/nrt'. (This is the first run.)


There are several ways to query for CMR updates that occured during a given timeframe. Read on in the CMR Search documentation:

* https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#c-with-new-granules (Collections)
* https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#c-with-revised-granules (Collections)
* https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#g-production-date (Granules)
* https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#g-created-at (Granules)

The `created_at` parameter works for our purposes. It's a granule search parameter that returns the records ingested since the input timestamp.

In [7]:
params = {
    'scroll': "true",
    'page_size': 2000,
    'sort_key': "-start_date",
    'collection_concept_id': ccid, 
    'created_at': timestamp,
    # Limit results to coverage for .5deg bbox in Gulf of Alaska:
    'bounding_box': "-146.5,57.5,-146,58",
}

params

{'scroll': 'true',
 'page_size': 2000,
 'sort_key': '-start_date',
 'collection_concept_id': 'C1968980576-POCLOUD',
 'created_at': '2022-11-08T00:15:46Z',
 'bounding_box': '-146.5,57.5,-146,58'}

Get the query parameters as a string and then the complete search url:

In [8]:
query = urlencode(params)
url = f"https://{cmr}/search/granules.umm_json?{query}"
print(url)

https://cmr.earthdata.nasa.gov/search/granules.umm_json?scroll=true&page_size=2000&sort_key=-start_date&collection_concept_id=C1968980576-POCLOUD&created_at=2022-11-08T00%3A15%3A46Z&bounding_box=-146.5%2C57.5%2C-146%2C58


Get a new timestamp that represents the UTC time of the search. Then download the records in `umm_json` format for granules that match our search parameters:

In [9]:
with urlopen(url) as f:
    results = loads(f.read().decode())

print(f"{results['hits']} new granules ingested for '{ccid}' since '{timestamp}'.")

timestamp = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")

0 new granules ingested for 'C1968980576-POCLOUD' since '2022-11-08T00:15:46Z'.


Neatly print the first granule record (if one was returned):

In [10]:
if len(results['items'])>0:
    print(dumps(results['items'][0], indent=2))

The link for http access can be retrieved from each granule record's `RelatedUrls` field. The download link is identified by `"Type": "GET DATA"` .

Select the download URL for each of the granule records:

In [11]:
downloads = [[u['URL'] for u in r['umm']['RelatedUrls'] if u['Type']=="GET DATA"][0] for r in results['items']]
downloads

[]

Finish by downloading the files to the data directory in a loop. Overwrite `.update` with a new timestamp on success.

In [12]:
for f in downloads:
    try:
        urlretrieve(f, f"{data}/{basename(f)}")
    except Exception as e:
        print(f"[{datetime.now()}] FAILURE: {f}\n\n{e}\n")
        raise e
    else:
        print(f"[{datetime.now()}] SUCCESS: {f}")

If there were updates to the local time series during this run and no exceptions were raised during the download loop, then overwrite the timestamp file that tracks updates to the data folder (`resources/nrt/.update`):

In [13]:
if len(results['items'])>0:
    with open(f"{data}/.update", "w") as f:
        f.write(timestamp)